# Dados de voos da ANAC

In [1]:
import pandas as pd

df = pd.read_csv('voos.csv')
df0 = df

print(df.shape)
df.head()

(783256, 13)


C:\Users\Diogo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa
0,0,AAL,1265,0,I,KMIA,SBEG,01/01/2018 20:30,01/01/2018 20:30,02/01/2018 01:53,02/01/2018 01:53,REALIZADO,NaN
1,1,AAL,1265,0,I,KMIA,SBEG,02/01/2018 20:30,02/01/2018 20:30,03/01/2018 01:53,03/01/2018 01:53,REALIZADO,NaN
2,2,AAL,1265,0,I,KMIA,SBEG,03/01/2018 20:30,03/01/2018 20:30,04/01/2018 01:53,04/01/2018 01:53,REALIZADO,NaN
3,3,AAL,1265,0,I,KMIA,SBEG,04/01/2018 20:30,04/01/2018 20:30,05/01/2018 01:53,05/01/2018 01:53,REALIZADO,NaN
4,4,AAL,1265,0,I,KMIA,SBEG,05/01/2018 20:30,05/01/2018 20:30,06/01/2018 01:53,06/01/2018 01:53,REALIZADO,NaN


In [2]:
df = df0

partida_sp = df['ICAO Aeródromo Origem'] == "SBGR"
chegada_rj = df['ICAO Aeródromo Destino']  == "SBRJ"
partida_rj = df['ICAO Aeródromo Origem'] == "SBRJ"
chegada_sp = df['ICAO Aeródromo Destino'] == "SBGR"

#df = df[(partida_sp & chegada_rj) | (partida_rj & chegada_sp)]
df = df[partida_sp & chegada_rj]


subregional = df['Código Tipo Linha'] == "H"
nacional = df['Código Tipo Linha'] == "N"
regional = df['Código Tipo Linha'] == "R"
df = df[nacional | regional | subregional]

df = df[df['Situação Voo']=='REALIZADO']
df.dropna(subset=['Chegada Prevista', 'Chegada Real', 'Partida Prevista'], inplace = True)

fmt='%d/%m/%Y %H:%M'
df['Partida Prevista'] = pd.to_datetime(df['Partida Prevista'], format=fmt)
df['Chegada Prevista'] = pd.to_datetime(df['Chegada Prevista'], format=fmt)
df['Chegada Real'] = pd.to_datetime(df['Chegada Real'], format=fmt)
df['Atraso'] = df['Chegada Real'] - df['Chegada Prevista']

#df['Hora Partida'] = df['Partida Prevista'].tm_hour
#df['Hora Partida'] = df['Partida Prevista'].apply(lambda x: int(x.hour))
df['Hora Partida'] = df['Partida Prevista'].apply(lambda x: x.hour + x.minute / 60)
df['Dia Mês'] = df['Partida Prevista'].apply(lambda x: int(x.day))


meses_ano = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
def str_mes(numero):    
    return meses_ano[numero-1]
    #return numero

df['Mês'] = df['Partida Prevista'].apply(lambda x: str_mes(int(x.month)))


dias_semana = ["Segunda", "Terca", "Quarta", "Quinta", "Sexta", "Sabado", "Domingo"]
def str_dia(numero):    
    return dias_semana[numero]

df['Dia Semana'] = df['Partida Prevista'].apply(lambda x: str_dia(int(x.weekday())))



df.Atraso = df['Atraso'].apply(lambda x: int(x.total_seconds()/60))

df1 = df

print(df.shape)
df.head()

(2599, 18)


,Unnamed: 0,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa,Atraso,Hora Partida,Dia Mês,Mês,Dia Semana
4326,4326,AZU,2400,0,R,SBGR,SBRJ,2018-01-01 06:15:00,01/01/2018 06:45,2018-01-01 07:15:00,2018-01-01 07:36:00,REALIZADO,WT,21,6.25,1,Janeiro,Segunda
4327,4327,AZU,2400,0,R,SBGR,SBRJ,2018-01-02 06:15:00,02/01/2018 06:03,2018-01-02 07:15:00,2018-01-02 07:11:00,REALIZADO,HD,-4,6.25,2,Janeiro,Terca
4328,4328,AZU,2400,0,R,SBGR,SBRJ,2018-01-03 06:15:00,03/01/2018 05:55,2018-01-03 07:15:00,2018-01-03 07:12:00,REALIZADO,HD,-3,6.25,3,Janeiro,Quarta
4329,4329,AZU,2400,0,R,SBGR,SBRJ,2018-01-04 06:15:00,04/01/2018 06:04,2018-01-04 07:15:00,2018-01-04 07:44:00,REALIZADO,HD,29,6.25,4,Janeiro,Quinta
4330,4330,AZU,2400,0,R,SBGR,SBRJ,2018-01-05 06:15:00,05/01/2018 06:07,2018-01-05 07:15:00,2018-01-05 07:04:00,REALIZADO,HD,-11,6.25,5,Janeiro,Sexta


# Dados meteorológicos de BDM

In [3]:
csv_files = ['lsestacoes_SBGR_18_01.csv',
             'lsestacoes_SBGR_18_02.csv',
             'lsestacoes_SBGR_18_03.csv',
             'lsestacoes_SBGR_18_04.csv',
             'lsestacoes_SBGR_18_05.csv',
             'lsestacoes_SBGR_18_06.csv',
             'lsestacoes_SBGR_18_07.csv',
             'lsestacoes_SBGR_18_08.csv',
             'lsestacoes_SBGR_18_09.csv']

fmt='%d/%m/%Y %H:%M'

df_tempo = pd.DataFrame()

for csv_file in csv_files: 
    print(csv_file)
    df_append = pd.read_csv(csv_file, encoding='iso-8859-1', delimiter=';')
    df_tempo = df_tempo.append(df_append)

df_tempo['Data/Hora'] = pd.to_datetime(df_tempo['Data/Hora'], format=fmt)

display(df_tempo)

lsestacoes_SBGR_18_01.csv
lsestacoes_SBGR_18_02.csv
lsestacoes_SBGR_18_03.csv
lsestacoes_SBGR_18_04.csv
lsestacoes_SBGR_18_05.csv
lsestacoes_SBGR_18_06.csv
lsestacoes_SBGR_18_07.csv
lsestacoes_SBGR_18_08.csv
lsestacoes_SBGR_18_09.csv


,Data/Hora,Tipo da nuvem da primeira camada,DireÇõÇo do Vento (0-360),Veloc. do Vento (m/s),Alt. da base das nuvens na primeira camada (m),Cobertura de nuvens da camada mais restrita(%),Cobertura de nuvens da primeira camada (oitavos),PressÇo reduzida ao nÇðvel do mar - QNH( hPa),Temp. do ar (C),Temp. do Ponto de Orvalho (C),Visibilidade horizontal (m) (acima de 10000m VV=10000),"Tempo Presente (Tabela 020003) <a href=""http://bancodedados.cptec.inpe.br/~rbanco/bdm/tabela_bufr_20003.pdf"">Detalhes</a>"
0,2018-01-01 00:00:00,0.0,320.0,3.60,206.0,2.0,656.168,1012.88,22.0,20.0,10000.0,0.0
1,2018-01-01 01:00:00,0.0,290.0,2.57,203.0,6.0,1.968.504,1014.91,22.0,20.0,10000.0,0.0
2,2018-01-01 02:00:00,0.0,320.0,3.60,172.0,3.0,984.252,1014.91,21.0,20.0,10000.0,0.0
3,2018-01-01 03:00:00,0.0,350.0,2.57,203.0,6.0,1.968.504,1014.91,22.0,20.0,10000.0,0.0
4,2018-01-01 04:00:00,0.0,350.0,3.09,202.0,6.0,1.968.504,1013.89,21.0,20.0,10000.0,0.0
5,2018-01-01 05:00:00,0.0,320.0,2.06,202.0,6.0,1.968.504,1013.89,21.0,20.0,10000.0,0.0
6,2018-01-01 06:00:00,0.0,330.0,2.57,173.0,6.0,1.968.504,1013.89,21.0,20.0,10000.0,0.0
7,2018-01-01 07:00:00,0.0,330.0,4.12,202.0,3.0,984.252,1012.88,21.0,20.0,10000.0,0.0
8,2018-01-01 08:00:00,0.0,310.0,2.06,156.0,2.0,656.168,1013.89,20.0,20.0,10000.0,0.0
9,2018-01-01 09:00:00,0.0,310.0,1.54,152.0,3.0,984.252,1013.89,21.0,20.0,10000.0,0.0


In [4]:
df = df1

#df = df[df['Mês']=='Janeiro']
df = pd.merge_asof(df.sort_values(by='Partida Prevista',ascending=True), df_tempo, left_on='Partida Prevista', right_on='Data/Hora')

df2 = df

display(df)

,Unnamed: 0,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,...,DireÇõÇo do Vento (0-360),Veloc. do Vento (m/s),Alt. da base das nuvens na primeira camada (m),Cobertura de nuvens da camada mais restrita(%),Cobertura de nuvens da primeira camada (oitavos),PressÇo reduzida ao nÇðvel do mar - QNH( hPa),Temp. do ar (C),Temp. do Ponto de Orvalho (C),Visibilidade horizontal (m) (acima de 10000m VV=10000),"Tempo Presente (Tabela 020003) <a href=""http://bancodedados.cptec.inpe.br/~rbanco/bdm/tabela_bufr_20003.pdf"">Detalhes</a>"
0,4326,AZU,2400,0,R,SBGR,SBRJ,2018-01-01 06:15:00,01/01/2018 06:45,2018-01-01 07:15:00,...,330.0,2.57,173.0,6.0,1.968.504,1013.89,21.0,20.0,10000.0,0.0
1,38882,GLO,1076,0,R,SBGR,SBRJ,2018-01-01 08:05:00,01/01/2018 08:05,2018-01-01 08:55:00,...,310.0,2.06,156.0,2.0,656.168,1013.89,20.0,20.0,10000.0,0.0
2,95763,TAM,4605,0,R,SBGR,SBRJ,2018-01-01 08:40:00,01/01/2018 08:40,2018-01-01 09:45:00,...,310.0,2.06,156.0,2.0,656.168,1013.89,20.0,20.0,10000.0,0.0
3,38943,GLO,1078,0,R,SBGR,SBRJ,2018-01-01 10:15:00,01/01/2018 10:15,2018-01-01 11:15:00,...,310.0,4.63,172.0,3.0,984.252,1013.89,22.0,20.0,10000.0,0.0
4,4353,AZU,2402,0,R,SBGR,SBRJ,2018-01-01 13:45:00,01/01/2018 13:37,2018-01-01 14:40:00,...,330.0,5.15,173.0,6.0,1.968.504,1014.91,24.0,20.0,10000.0,0.0
5,39028,GLO,1082,0,R,SBGR,SBRJ,2018-01-01 14:35:00,01/01/2018 14:35,2018-01-01 15:40:00,...,300.0,4.12,213.0,6.0,1.968.504,1014.91,25.0,20.0,10000.0,0.0
6,39062,GLO,1084,0,R,SBGR,SBRJ,2018-01-01 17:25:00,01/01/2018 17:46,2018-01-01 18:25:00,...,320.0,5.66,302.0,3.0,984.252,1012.88,27.0,21.0,10000.0,0.0
7,39124,GLO,1086,0,R,SBGR,SBRJ,2018-01-01 18:25:00,01/01/2018 18:25,2018-01-01 19:25:00,...,320.0,5.15,302.0,3.0,984.252,1012.88,28.0,21.0,10000.0,0.0
8,39153,GLO,1088,0,R,SBGR,SBRJ,2018-01-01 19:25:00,01/01/2018 19:25,2018-01-01 20:25:00,...,300.0,6.18,352.0,3.0,984.252,1010.85,28.0,20.0,10000.0,0.0
9,22212,AZU,5016,0,N,SBGR,SBRJ,2018-01-01 20:05:00,01/01/2018 20:05,2018-01-01 21:05:00,...,310.0,4.12,382.0,3.0,984.252,1010.85,28.0,21.0,10000.0,0.0


In [5]:
from datetime import datetime

df = df2


''
categorias = ['ICAO Empresa Aérea',
              'ICAO Aeródromo Origem',
              'ICAO Aeródromo Destino',
              'Número Voo',
              'Código Autorização (DI)',
              'Código Tipo Linha',
              'Código Justificativa',
              'Situação Voo',
              'Dia Mês',
              'Mês',
              'Dia Semana']
for cat in categorias:
    df[cat] = df[cat].astype('category')

    
df = df.drop(columns=['Unnamed: 0',
                      'Data/Hora',
                      #'Número Voo',
                      'Código Autorização (DI)',
                      'Código Tipo Linha',
                      #'Situação Voo',
                      'Partida Prevista',
                      'Partida Real',
                      'Chegada Prevista',
                      'Chegada Real',
                      'Cobertura de nuvens da primeira camada (oitavos)',
                      'ICAO Aeródromo Origem',
                      'ICAO Aeródromo Destino',
                      'Código Justificativa'])

df3 = df

print(df.shape)
#df.head()
display(df)

(2599, 18)


,ICAO Empresa Aérea,Número Voo,Situação Voo,Atraso,Hora Partida,Dia Mês,Mês,Dia Semana,Tipo da nuvem da primeira camada,DireÇõÇo do Vento (0-360),Veloc. do Vento (m/s),Alt. da base das nuvens na primeira camada (m),Cobertura de nuvens da camada mais restrita(%),PressÇo reduzida ao nÇðvel do mar - QNH( hPa),Temp. do ar (C),Temp. do Ponto de Orvalho (C),Visibilidade horizontal (m) (acima de 10000m VV=10000),"Tempo Presente (Tabela 020003) <a href=""http://bancodedados.cptec.inpe.br/~rbanco/bdm/tabela_bufr_20003.pdf"">Detalhes</a>"
0,AZU,2400,REALIZADO,21,6.250000,1,Janeiro,Segunda,0.0,330.0,2.57,173.0,6.0,1013.89,21.0,20.0,10000.0,0.0
1,GLO,1076,REALIZADO,0,8.083333,1,Janeiro,Segunda,0.0,310.0,2.06,156.0,2.0,1013.89,20.0,20.0,10000.0,0.0
2,TAM,4605,REALIZADO,0,8.666667,1,Janeiro,Segunda,0.0,310.0,2.06,156.0,2.0,1013.89,20.0,20.0,10000.0,0.0
3,GLO,1078,REALIZADO,0,10.250000,1,Janeiro,Segunda,0.0,310.0,4.63,172.0,3.0,1013.89,22.0,20.0,10000.0,0.0
4,AZU,2402,REALIZADO,-3,13.750000,1,Janeiro,Segunda,0.0,330.0,5.15,173.0,6.0,1014.91,24.0,20.0,10000.0,0.0
5,GLO,1082,REALIZADO,0,14.583333,1,Janeiro,Segunda,0.0,300.0,4.12,213.0,6.0,1014.91,25.0,20.0,10000.0,0.0
6,GLO,1084,REALIZADO,21,17.416667,1,Janeiro,Segunda,0.0,320.0,5.66,302.0,3.0,1012.88,27.0,21.0,10000.0,0.0
7,GLO,1086,REALIZADO,0,18.416667,1,Janeiro,Segunda,0.0,320.0,5.15,302.0,3.0,1012.88,28.0,21.0,10000.0,0.0
8,GLO,1088,REALIZADO,0,19.416667,1,Janeiro,Segunda,0.0,300.0,6.18,352.0,3.0,1010.85,28.0,20.0,10000.0,0.0
9,AZU,5016,REALIZADO,0,20.083333,1,Janeiro,Segunda,0.0,310.0,4.12,382.0,3.0,1010.85,28.0,21.0,10000.0,0.0


In [6]:
df.to_csv('voos_GRU_SDU.csv')